In [1]:
import sys
sys.path.append('./')
import numpy as np
import pandas as pd
import neural_tape_controller
#Положительные числа - положительные награды.
import tasks 
import optimize
import pickle
root_dir = './'

In [2]:
nt = neural_tape_controller.nt_controller(tacts=1,input_size=36,output_size=5)
genom = nt.nn.disassemble_genom()

In [3]:
try:
    with open(root_dir+f'./genoms/best_genoms_aa_gun2.pkl', 'rb') as f:
        genom = pickle.load(f)[-1]
    print('loaded successfully')
except Exception:
    pass

loaded successfully


In [3]:
%%time
q=tasks.custom_test(genom, verbose=True, draw=False)
print('q=',q)

/home/nikita/neuroevolution-toolbox/nnet.py:316: RuntimeWarning: overflow encountered in add
  self.belts[layer['belt_name']][:,:min_len] += in_data[:,:min_len]


aa_gun_2: [-218.8899  -88.9622 -188.901  -203.9799 -118.9979 -103.9169]  full=-205.77344499999998
q= -205.7734450000001
CPU times: user 15.5 s, sys: 49 s, total: 1min 4s
Wall time: 5.57 s


In [3]:
print(pd.Timestamp.now())

2022-05-15 09:57:03.565596


In [5]:
%%time
n_jobs = 12
from multiprocessing import Pool
pool = Pool(processes=n_jobs)
y_lst = pool.map(tasks.custom_test, [x for x in [genom]*n_jobs])
pool.close()
pool.join()

CPU times: user 53.1 ms, sys: 37.5 ms, total: 90.6 ms
Wall time: 1min 3s


In [4]:
print(1)
opt = optimize.optimizer(tasks.custom_test, genom_size=len(genom),parallel_cores=12)

1


In [6]:
try:
    with open(root_dir+f'./genoms/best_genoms_aa_gun.pkl', 'rb') as f:
        opt.best_genoms = pickle.load(f)
    print('loaded successfully')
except Exception:
    pass

loaded successfully


In [5]:
opt.function(opt.best_genoms[-1], verbose=True)
for i in range(1000):
    print('opt#',i)
    opt.optimize()
    with open(root_dir+f'./genoms/best_genoms_aa_gun2.pkl', 'wb') as f:
        pickle.dump(opt.best_genoms,f,protocol=pickle.HIGHEST_PROTOCOL)
        print('WRITTEN')

/home/nikita/neuroevolution-toolbox/nnet.py:324: RuntimeWarning: overflow encountered in multiply
  k_add = np.arctan(k_amplif*self.belts[layer['belt_name']][:,:min_len]/threshold)*threshold


aa_gun_2: [-218.9637 -188.9946 -188.9547 -203.9928 -218.9913 -188.9214]  full=-215.4867554
opt# 0
scores for optimizers augmented [1.e+10 1.e+10 1.e+10 1.e+10 1.e+10 1.e+10 1.e+10 1.e+10]
chosen evol_wide previous_result: nan per tacts: nan


/home/nikita/neuroevolution-toolbox/optimize.py:36: RuntimeWarning: Mean of empty slice
  mx.append(np.nanmean(self.history_gain[opt_name])-time_penalty*np.nanmean(self.history_time[opt_name]))
/home/nikita/.local/lib/python3/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


iteration 0 y= [-163.54710337 -170.22726333 -174.20432333 -174.7620543 ]
iteration 1 y= [-163.54710337 -169.87967692 -170.22726333 -174.20432333]
iteration 2 y= [-159.15597667 -163.54710337 -166.77038    -169.87967692]
iteration 3 y= [-159.15597667 -163.54710337 -166.77038    -169.87967692]
iteration final y= [-135.8016184  -158.15088437 -158.4217667  -159.15597667]
result evol_wide previous_gain: nan per tacts: nan duration 0 days 00:24:22.063841
aa_gun_2: [  80.1923  225.2373  -74.8818  411.2766    2.0846 -189.8914]  full=-135.8016184
WRITTEN
opt# 1
scores for optimizers augmented [2.7645485e+01 1.0000000e+10 1.0000000e+10 1.0000000e+10 1.0000000e+10
 1.0000000e+10 1.0000000e+10 1.0000000e+10]
chosen evol_mid_chaos previous_result: nan per tacts: nan
iteration 0 y= [-135.8016184  -177.43333333 -177.73190333]
iteration 1 y= [-135.8016184  -177.35816667 -177.3915    ]
iteration final y= [-135.8016184 -176.5       -176.5      ]
result evol_mid_chaos previous_gain: nan per tacts: nan dur

In [ ]:
#draw
with open(root_dir+f'./genoms/best_genoms_aa_gun.pkl', 'rb') as f:
    genom = pickle.load(f)
    genom = genom[-1]
print('loaded successfully')
q=tasks.custom_test(genom, verbose=True, draw=True)
print('q=',q)

In [ ]:
#save gif
tasks.video[0].save(
    root_dir+'./out_videos/best_genoms_aa_gun.gif',
    save_all=True,
    append_images=tasks.video[1:], 
    optimize=True,
    duration=200,
    loop=0
)

In [9]:
#draw video
import cv2
writer = cv2.VideoWriter("outputvideo.mp4",cv2.VideoWriter_fourcc(*"MP4V"),30,(512,512))
for i in range(0,len(tasks.video),1):
    resize = cv2.resize(np.array(tasks.video[i]), (512, 512))
    frame = resize*0
    frame[:,:,0],frame[:,:,1],frame[:,:,2] = resize[:,:,2],resize[:,:,1], resize[:,:,0]
    writer.write(frame)
writer.release()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
